<a href="https://colab.research.google.com/github/guilhermevaz8/RL_Taxi_Problem/blob/main/MaskPPO_Taxi_5M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://cdn.comet.ml/img/notebook_logo.png">  <img height=90px align="right" src="https://raw.githubusercontent.com/Farama-Foundation/Gymnasium/main/gymnasium-text.png">

[Comet](https://www.comet.com/?utm_source=gymnasium&utm_medium=partner&utm_campaign=partner_gymnasium_2023&utm_content=comet_colab) helps accelerate machine learning through experiment tracking and production monitoring. Never lose track of your work and easily share results and collaborate across teams of all sizes.

**This notebook shows you how to log your Gymnasium metrics with Comet.** For more information about Comet's integration with Gymnasium, visit our [Docs](https://www.comet.com/docs/v2/integrations/ml-frameworks/gymnasium/?utm_source=gymnasium&utm_medium=partner&utm_campaign=partner_gymnasium_2023&utm_content=comet_colab) page.

If you prefer to preview what's to come, check out completed experiments created from this notebook [here](https://www.comet.com/examples/comet-examples-gymnasium-notebook/?utm_source=gymnasium&utm_medium=partner&utm_campaign=partner_gymnasium_2023&utm_content=comet_colab).



# Install Required Packages

In [ ]:
%pip install gymnasium comet_ml stable-baselines3 sb3_contrib

# Initialize Comet

Create your free account at [Comet.com](https://www.comet.com/signup?utm_source=gymnasium&utm_medium=partner&utm_campaign=partner_gymnasium_2023&utm_content=comet_colab) and grab your API key which can be found under account settings.

In [ ]:
import comet_ml

comet_ml.init(project_name="comet-examples-gymnasium-notebook")

# Train an Agent using StableBaselines3 A2C Algorithm with DummyVecEnv

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

from comet_ml.integration.gymnasium import CometLogger
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO


def mask_fn(env):
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.unwrapped.action_mask(env.unwrapped.s)


env = gym.make("Taxi-v3")


#env = gym.wrappers.RecordVideo(env, 'test')

env = ActionMasker(env, mask_fn)  # Wrap to enable masking


experiment = comet_ml.Experiment()

env = CometLogger(env, experiment)

# MaskablePPO behaves the same as SB3's PPO unless the env is wrapped
# with ActionMasker. If the wrapper is detected, the masks are automatically
# retrieved and used when learning. Note that MaskablePPO does not accept
# a new action_mask_fn kwarg, as it did in an earlier draft.
model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=0)
model.learn(total_timesteps=5_000_000, progress_bar=True)
model.save("maskppo_taxi_5M")

env.close()
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/campiao/comet-examples-gymnasium-notebook/81d96b1935d24332bd988fca45179fc3



Output()

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables
from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do 
`env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search 
the reminding wrappers.
  logger.warn(

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/campiao/comet-examples-gymnasium-notebook/81d96b1935d24332bd988fca45179fc3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     episode_length [339270] : (6, 200)
COMET INFO:     episode_reward [339270] : (-200, 15)
COMET INFO:     loss [78144]            : (-0.06284510344266891, 157.0519561767578)
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
─────────────────────────────────────────────── New Com